In [187]:
import pandas as pd
import numpy as np

In [188]:
df = pd.read_csv('../../data/raw/train.csv')
df = df.drop('ID', axis = 1)
df = df.loc[df['배란 자극 여부'] == 1,:] # 배란 자극을 한 사람만 포함
df = df.loc[df['시술 유형'] != 'DI',:] # DI 시술인 사람은 제외
df = df.loc[(df['PGD 시술 여부'] != 1) | (df['PGS 시술 여부'] != 1),:] # PGD, PGS검사를 받은 사람은 제외

# 혼합할 만한 난자

In [189]:
df[(df["수집된 신선 난자 수"] + df['해동 난자 수'])==0]['임신 성공 여부'].value_counts()

0    2284
1       6
Name: 임신 성공 여부, dtype: int64

In [190]:
df = df[(df["수집된 신선 난자 수"] + df['해동 난자 수'])!=0]
print(df['난자 출처'].unique())

['본인 제공']


In [192]:
# 혼합된 난자 수 대비 수집된 난자 수 비율 계산
df["혼합할만한 난자 비율"] = (df['저장된 신선 난자 수'] + df['혼합된 난자 수']) / (df["수집된 신선 난자 수"] + df['해동 난자 수'])

check = df[df['혼합할만한 난자 비율']>1]
display(check)
# 비율이 1보다 큰 환자 3명 존재 -> 이상치

df = df[df['혼합할만한 난자 비율']<=1]

# 비율 구간화 (기준은 임의 설정, 필요시 조정 가능)
df["혼합할만한 난자 비율 구간"] = pd.cut(
    df["혼합할만한 난자 비율"],
    bins=[0, 0.3, 0.6, 1.0],  # 임의로 설정 가능
    labels=["낮음", "중간", "높음"],
    include_lowest=True
)

,시술 시기 코드,시술 당시 나이,임신 시도 또는 마지막 임신 경과 연수,시술 유형,특정 시술 유형,배란 자극 여부,배란 유도 유형,단일 배아 이식 여부,착상 전 유전 검사 사용 여부,착상 전 유전 진단 사용 여부,...,대리모 여부,PGD 시술 여부,PGS 시술 여부,난자 채취 경과일,난자 해동 경과일,난자 혼합 경과일,배아 이식 경과일,배아 해동 경과일,임신 성공 여부,혼합할만한 난자 비율


In [193]:
display(df['혼합할만한 난자 비율 구간'].value_counts())

success_counts = df.groupby('혼합할만한 난자 비율 구간')['임신 성공 여부'].value_counts().unstack()
display(success_counts)
success_rates = (success_counts[1] / (success_counts[0] + success_counts[1])) * 100
print(success_rates)

높음    171999
중간     17184
낮음      2958
Name: 혼합할만한 난자 비율 구간, dtype: int64

임신 성공 여부,0,1
혼합할만한 난자 비율 구간,,
낮음,2769,189
중간,13150,4034
높음,124040,47959


혼합할만한 난자 비율 구간
낮음     6.389452
중간    23.475326
높음    27.883302
dtype: float64


# 배아 생성 비율

In [194]:
df['배아 생성 비율'] = df['총 생성 배아 수']/df['혼합된 난자 수']
print((df['배아 생성 비율'] > 1).sum())
df = df[df['배아 생성 비율'] <= 1] # 배아 생성 비율이 1보다 크면 drop (2행)

df['배아 생성 비율 구간'] = pd.qcut(df["배아 생성 비율"], q=3, labels=["낮음", "중간", "높음"])

display(df['배아 생성 비율 구간'].value_counts())

success_counts = df.groupby('배아 생성 비율 구간')['임신 성공 여부'].value_counts().unstack()
display(success_counts)
success_rates = (success_counts[1] / (success_counts[0] + success_counts[1])) * 100
print(success_rates)

2


중간    68982
낮음    63922
높음    55270
Name: 배아 생성 비율 구간, dtype: int64

임신 성공 여부,0,1
배아 생성 비율 구간,,
낮음,50080,13842
중간,47842,21140
높음,38071,17199


배아 생성 비율 구간
낮음    21.654516
중간    30.645676
높음    31.118147
dtype: float64


# 착상할만한 배아 수

In [195]:
df['착상할만한 배아 생성 비율'] = (df['이식된 배아 수'] + df['저장된 배아 수'])/df['총 생성 배아 수']
df['착상할만한 배아 생성 비율'] = df['착상할만한 배아 생성 비율'].replace([np.inf, -np.inf], 0)
print((df['착상할만한 배아 생성 비율']>1).sum())
df = df[(df['착상할만한 배아 생성 비율'] <= 1)] # 쓸만한 배아 생성 비율이 1보다 큰 행 제거 (187행)

df['착상할만한 배아 생성 비율 구간'] = pd.qcut(df["착상할만한 배아 생성 비율"], q=3, labels=["낮음", "중간", "높음"])

display(df['착상할만한 배아 생성 비율 구간'].value_counts())

success_counts = df.groupby('착상할만한 배아 생성 비율 구간')['임신 성공 여부'].value_counts().unstack()
display(success_counts)
success_rates = (success_counts[1] / (success_counts[0] + success_counts[1])) * 100
print(success_rates)

187


중간    69631
낮음    61495
높음    51339
Name: 착상할만한 배아 생성 비율 구간, dtype: int64

임신 성공 여부,0,1
착상할만한 배아 생성 비율 구간,,
낮음,42642,18853
중간,47179,22452
높음,40496,10843


착상할만한 배아 생성 비율 구간
낮음    30.657777
중간    32.244259
높음    21.120396
dtype: float64


In [198]:
# 그룹별 개수 계산
grouped_counts = df.groupby(['혼합할만한 난자 비율 구간', '배아 생성 비율 구간', '착상할만한 배아 생성 비율 구간'])['임신 성공 여부'].value_counts().unstack()
# '임신 성공 여부' 비율 계산 (각 그룹 내에서 성공(1)의 비율)
success_ratio = grouped_counts.div(grouped_counts.sum(axis=1), axis=0) * 100  # 백분율 변환
grouped_counts['rate'] = success_ratio[1].round(3)
display(grouped_counts)

임신 성공 여부                                            0       1    rate
혼합할만한 난자 비율 구간 배아 생성 비율 구간 착상할만한 배아 생성 비율 구간                         
낮음             낮음          낮음                    18.0     NaN     NaN
                           중간                    21.0     6.0  22.222
                           높음                   185.0    24.0  11.483
               중간          낮음                    13.0     5.0  27.778
                           중간                    47.0    16.0  25.397
                           높음                    60.0    18.0  23.077
               높음          낮음                    43.0    15.0  25.862
                           중간                    74.0    29.0  28.155
                           높음                   351.0    76.0  17.799
중간             낮음          낮음                   468.0   132.0  22.000
                           중간                   875.0   370.0  29.719
                           높음                  2433.0   457.0  15.813
               중간          낮음                   961.0   414.0  30.109
                           중간                  1567.0   723.0  31.572
                           높음                  1182.0   340.0  22.339
               높음          낮음                  1091.0   560.0  33.919
                           중간                  1390.0   670.0  32.524
                           높음                  1663.0   361.0  17.836
높음             낮음          낮음                  9504.0  3440.0  26.576
                           중간                 13791.0  5494.0  28.488
                           높음                 17167.0  3895.0  18.493
               중간          낮음                 17460.0  7928.0  31.227
                           중간                 17156.0  8555.0  33.274
                           높음                  9369.0  3138.0  25.090
               높음          낮음                 13084.0  6359.0  32.706
                           중간                 12258.0  6589.0  34.960
                           높음                  8086.0  2534.0  23.861

In [199]:
# 그룹별 개수 계산
grouped_counts = df.groupby(['혼합할만한 난자 비율 구간', '배아 생성 비율 구간'])['임신 성공 여부'].value_counts().unstack()
# '임신 성공 여부' 비율 계산 (각 그룹 내에서 성공(1)의 비율)
success_ratio = grouped_counts.div(grouped_counts.sum(axis=1), axis=0) * 100  # 백분율 변환
grouped_counts['rate'] = success_ratio[1].round(3)
display(grouped_counts)

임신 성공 여부                        0      1    rate
혼합할만한 난자 비율 구간 배아 생성 비율 구간                      
낮음             낮음             224     30  11.811
               중간             120     39  24.528
               높음             468    120  20.408
중간             낮음            3776    959  20.253
               중간            3710   1477  28.475
               높음            4144   1591  27.742
높음             낮음           40462  12829  24.073
               중간           43985  19621  30.848
               높음           33428  15482  31.654

# IVF 성공률

In [213]:
df = pd.read_csv('../../data/raw/train.csv')
df = df.drop('ID', axis = 1)
df = df.loc[df['배란 자극 여부'] == 1,:] # 배란 자극을 한 사람만 포함
df = df.loc[df['시술 유형'] != 'DI',:] # DI 시술인 사람은 제외
df = df.loc[(df['PGD 시술 여부'] != 1) | (df['PGS 시술 여부'] != 1),:] # PGD, PGS검사를 받은 사람은 제외

In [214]:
df.groupby(['IVF 시술 횟수', 'IVF 임신 횟수', 'IVF 출산 횟수']).size()

IVF 시술 횟수  IVF 임신 횟수  IVF 출산 횟수
0회         0회         0회           94426
1회         0회         0회           36397
           1회         0회            1961
                      1회            4461
2회         0회         0회           19544
           1회         0회            1964
                      1회            4578
           2회         0회              75
                      1회             228
                      2회             119
3회         0회         0회            8961
           1회         0회            1309
                      1회            3240
           2회         0회              83
                      1회             334
                      2회             186
           3회         0회               3
                      1회              12
                      2회               7
                      3회               4
4회         0회         0회            4194
           1회         0회             813
                      1회            1990
           2회         0회 

In [228]:
numeric_IVF = df[['IVF 시술 횟수', 'IVF 임신 횟수', 'IVF 출산 횟수']].apply(lambda x : x.str[0].astype(int))
df['IVF 성공 확률'] =  numeric_IVF['IVF 임신 횟수'] / numeric_IVF['IVF 시술 횟수'].replace(0, np.nan)

# 함수화

In [ ]:
import pandas as pd
import numpy as np

def filter_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    특정 조건을 만족하는 데이터를 필터링하는 함수.
    - 배란 자극을 하지 않은 사람 제거
    - DI 시술을 한 사람 제거
    - PGD/PGS 검사를 받은 사람 제거
    
    Parameters:
        df (pd.DataFrame): 원본 데이터프레임
    
    Returns:
        pd.DataFrame: 필터링된 데이터프레임
    """
    df = df.drop('ID', axis=1, errors='ignore')  # ID 컬럼이 있으면 삭제
    df = df[df['배란 자극 여부'] == 1]  # 배란 자극을 한 사람만 포함
    df = df[df['시술 유형'] != 'DI']  # DI 시술을 한 사람 제외
    df = df[(df['PGD 시술 여부'] != 1) | (df['PGS 시술 여부'] != 1)]  # PGD, PGS 검사를 받은 사람 제외
    return df


def add_oocyte_ratio(df: pd.DataFrame, remove_outliers=True) -> pd.DataFrame:
    """
    혼합할만한 난자 비율을 계산하고 구간화를 수행하는 함수.
    
    Parameters:
        df (pd.DataFrame): 데이터프레임
        remove_outliers (bool): 비율이 1보다 큰 행을 제거할지 여부
    
    Returns:
        pd.DataFrame: 혼합할만한 난자 비율이 추가된 데이터프레임
    """
    df = df[(df["수집된 신선 난자 수"] + df['해동 난자 수']) != 0].copy()  # 0인 경우 제외

    # 혼합할만한 난자 비율 계산
    df["혼합할만한 난자 비율"] = (df['저장된 신선 난자 수'] + df['혼합된 난자 수']) / (df["수집된 신선 난자 수"] + df['해동 난자 수'])

    # 이상치 제거 옵션
    if remove_outliers:
        df = df[df['혼합할만한 난자 비율'] <= 1]

    # 비율 구간화
    df["혼합할만한 난자 비율 구간"] = pd.cut(
        df["혼합할만한 난자 비율"],
        bins=[0, 0.3, 0.6, 1.0],
        labels=["낮음", "중간", "높음"],
        include_lowest=True
    )
    
    return df


def add_embryo_creation_ratio(df: pd.DataFrame, remove_outliers=True) -> pd.DataFrame:
    """
    배아 생성 비율을 추가하고 구간화를 수행하는 함수.

    Parameters:
        df (pd.DataFrame): 데이터프레임
        remove_outliers (bool): 비율이 1보다 큰 행을 제거할지 여부
    
    Returns:
        pd.DataFrame: 배아 생성 비율이 추가된 데이터프레임
    """
    df['배아 생성 비율'] = df['총 생성 배아 수'] / df['혼합된 난자 수']
    
    # 이상치 제거 옵션
    if remove_outliers:
        df = df[df['배아 생성 비율'] <= 1]

    # 배아 생성 비율 구간화
    df['배아 생성 비율 구간'] = pd.qcut(df["배아 생성 비율"], q=3, labels=["낮음", "중간", "높음"])
    
    return df


def add_implantable_embryo_ratio(df: pd.DataFrame, remove_outliers=True) -> pd.DataFrame:
    """
    착상할만한 배아 생성 비율을 추가하고 구간화를 수행하는 함수.

    Parameters:
        df (pd.DataFrame): 데이터프레임
        remove_outliers (bool): 비율이 1보다 큰 행을 제거할지 여부
    
    Returns:
        pd.DataFrame: 착상할만한 배아 생성 비율이 추가된 데이터프레임
    """
    df['착상할만한 배아 생성 비율'] = (df['이식된 배아 수'] + df['저장된 배아 수']) / df['총 생성 배아 수']
    df['착상할만한 배아 생성 비율'] = df['착상할만한 배아 생성 비율'].replace([np.inf, -np.inf], 0)

    # 이상치 제거 옵션
    if remove_outliers:
        df = df[df['착상할만한 배아 생성 비율'] <= 1]

    # 착상할만한 배아 생성 비율 구간화
    df['착상할만한 배아 생성 비율 구간'] = pd.qcut(df["착상할만한 배아 생성 비율"], q=3, labels=["낮음", "중간", "높음"])
    
    return df


def add_ivf_success_rate(df: pd.DataFrame) -> pd.DataFrame:
    """
    IVF 성공 확률을 계산하는 함수.
    
    Parameters:
        df (pd.DataFrame): 원본 데이터프레임
    
    Returns:
        pd.DataFrame: IVF 성공 확률 컬럼이 추가된 데이터프레임
    """
    # IVF 관련 컬럼을 숫자로 변환 (첫 번째 문자만 추출 후 변환)
    numeric_IVF = df[['IVF 시술 횟수', 'IVF 임신 횟수', 'IVF 출산 횟수']].apply(
        lambda x: pd.to_numeric(x.str[0], errors='coerce')
    )

    # IVF 성공 확률 계산 (0으로 나누는 경우 NaN 처리)
    df['IVF 성공 확률'] = numeric_IVF['IVF 임신 횟수'] / numeric_IVF['IVF 시술 횟수'].replace(0, np.nan)

    return df


'''
# 데이터 불러오기
df = pd.read_csv('../../data/raw/train.csv')

# 1️⃣ 데이터 필터링 적용
df = filter_data(df)

# 2️⃣ 혼합할만한 난자 비율 추가 (이상치 제거 옵션 포함)
df = add_oocyte_ratio(df, remove_outliers=True)

# 3️⃣ 배아 생성 비율 추가 (이상치 제거 옵션 포함)
df = add_embryo_creation_ratio(df, remove_outliers=True)

# 4️⃣ 착상할만한 배아 생성 비율 추가 (이상치 제거 옵션 포함)
df = add_implantable_embryo_ratio(df, remove_outliers=True)

# 결과 확인
print(df.head())

'''